# Resume NER
## Extract Information from Resumes using NER (Named Entity Recognition)

### Part 2 - NER with Spacy
In this second part of the challenge, we will be using the preprocessed data from part one to start training NER models. We will be using spacy (https://spacy.io/) here to "get our feet wet" with NER, as training spacy can be reasonably done on our laptops and does not yet necessarily require a GPU. Spacy is a powerful, effective, and resource-efficient NLP library - It might surprise us with its performance on the challenge!

We will run spacy's pretrained models on our data to get a feel for NER, and then we will perform some additional preprocessing on our data before we start training our own NER model using the labelled entities we have identified in part one. 
We will also explore evaluation metrics for NER, and decide how we want to quantify the performance of our trained models. 

* *If you need help setting up python or running this notebook, please get help from the  assistants to the professor*
* *It might be helpful to try your code out first in a python ide like pycharm before copying it an running it here in this notebook*
* *For solving the programming tasks, use the python reference linked here (Help->Python Reference) as well as Web-searches.* 

##### Reload preprocessed data
Here, we will load the data we saved in part one and save it to a variable. Provide code below to load the data and store it as a list in a variable. (Hint - use 'open' and the json module)

In [0]:
## import json module
import json
path = 'data/converted_resumes.json'
## TODO open file load as json and store in "resumes" variable
with open(path, 'r') as f:
    resumes = json.load(f)
## TODO print length of loaded resumes list to be sure everything ok
print(len(resumes))

690


##### Take Spacy for a spin
Before we train our own NER model to recognize the resume-specific entities we want to capture, let's see how spacy's pretrained NER models do on our data. These pretrained models can't recognize our entities yet, but let's see how they do. Run the next code block to load spacy's English language model 


In [0]:
! pip install spacy

In [0]:
! python -m spacy download en
#! pip list

  Running setup.py install for en-core-web-sm: started
    Running setup.py install for en-core-web-sm: finished with status 'done'
[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
symbolic link created for C:\Users\nakami\Anaconda3\lib\site-packages\spacy\data\en <<===>> C:\Users\nakami\Anaconda3\lib\site-packages\en_core_web_sm
[+] Linking successful
C:\Users\nakami\Anaconda3\lib\site-packages\en_core_web_sm -->
C:\Users\nakami\Anaconda3\lib\site-packages\spacy\data\en
You can now load the model via spacy.load('en')


In [0]:
import spacy
nlp = spacy.load('en')
print(nlp)

Now we get the EntityRecognizer in the loaded nlp pipeline and display the labels it supports

In [0]:
ner = nlp.get_pipe('ner')
labels = ner.labels
print(labels)

('PERCENT', 'MONEY', 'CARDINAL', 'LOC', 'ORG', 'EVENT', 'LAW', 'LANGUAGE', 'ORDINAL', 'PERSON', 'DATE', 'FAC', 'PRODUCT', 'NORP', 'TIME', 'GPE', 'QUANTITY', 'WORK_OF_ART')


##### Question: What do the 'GPE', 'FAC' and 'NORP' labels stand for? (Tipp: use either the spacy.explain method, or google the spacy.io api docs) 
See below

In [0]:
### TODO  if you choose to use spacy's 'explain' method to get the answer to the question above, provide your code here
## print description of entities using spacy explain
for label in ['GPE', 'FAC', 'NORP']:
    print(f'{label}:\t{spacy.explain(label)}')
print()
# print(f'{"ORDINAL"}: {spacy.explain("ORDINAL")}')

GPE:	Countries, cities, states
FAC:	Buildings, airports, highways, bridges, etc.
NORP:	Nationalities or religious or political groups



As we can see, the entities are different than the entities we will train our custom model on. 
##### Question: what entities do you think this model will find in an example resume?
1. GPE: Countries, cities, states
2. LANGUAGE: Any named language
3. DATE: Absolute or relative dates or periods
4. PERSON: People, including fictional
5. ORG: People, including fictional
6. ORDINAL: "first", "second", etc.

Now we will work with one of our resumes, and get spacy to tell us what entities it recognizes. Complete the code block below to get a single resume text out of our resume list. 

In [0]:
### TODO get a single resume text and print it out
restxt = resumes[0][0]
## print it out, removing extraneous spaces
print("\n".join(restxt.split('\n\n')))

Afreen Jamadar
Active member of IIIT Committee in Third year
Sangli, Maharashtra - Email me on Indeed: indeed.com/r/Afreen-Jamadar/8baf379b705e37c6
I wish to use my knowledge, skills and conceptual understanding to create excellent team
environments and work consistently achieving organization objectives believes in taking initiative
and work to excellence in my work.
WORK EXPERIENCE
Active member of IIIT Committee in Third year
Cisco Networking -  Kanpur, Uttar Pradesh
organized by Techkriti IIT Kanpur and Azure Skynet.
PERSONALLITY TRAITS:
• Quick learning ability
• hard working
EDUCATION
PG-DAC
CDAC ACTS
2017
Bachelor of Engg in Information Technology
Shivaji University Kolhapur -  Kolhapur, Maharashtra
2016
SKILLS
Database (Less than 1 year), HTML (Less than 1 year), Linux. (Less than 1 year), MICROSOFT
ACCESS (Less than 1 year), MICROSOFT WINDOWS (Less than 1 year)
ADDITIONAL INFORMATION
TECHNICAL SKILLS:
• Programming Languages: C, C++, Java, .net, php.
• Web Designing: HTML, XML

Extracting entities with spacy is easy with a pretrained model. We simply call the model (here 'nlp') with our text to get a spacy Document. See https://spacy.io/api/doc for more detail. 

Execute the code below to process the resume txt.

In [0]:
doc = nlp(restxt)
doc.ents[0].label_

'GPE'

The doc object has a list of entities predicted by spacy 'ents'. We would like to loop through all of these entities and print their label and associated text to see what spacy predicted for this resume.

Complete the code below to do this. You will probably need to google the spacy api docs to find the solution (Tipp: look for 'Doc.ents'). Also, trying code in your ide (for example pycharm) before copying it here might help with exploring and debugging to find the solution. 

In [0]:
##TODO loop through the doc's entities, and print the label and text for each entity found. 
for index, ent in enumerate(doc.ents):
    print(f'{index}')
    print(f'\tlabel_:\t{ent.label_} ({ent.label})')
    print(f'\ttext:\t{ent.text}')

0
	label_:	GPE (384)
	text:	Jamadar
1
	label_:	ORG (383)
	text:	IIIT Committee
2
	label_:	DATE (391)
	text:	Third year
3
	label_:	ORG (383)
	text:	Maharashtra
4
	label_:	ORG (383)
	text:	IIIT Committee
5
	label_:	DATE (391)
	text:	Third year
6
	label_:	PERSON (380)
	text:	Cisco Networking -  Kanpur
7
	label_:	ORG (383)
	text:	Uttar Pradesh
8
	label_:	ORG (383)
	text:	Techkriti IIT Kanpur
9
	label_:	PERSON (380)
	text:	Azure Skynet
10
	label_:	PERSON (380)
	text:	Quick
11
	label_:	ORG (383)
	text:	CDAC
12
	label_:	DATE (391)
	text:	2017
13
	label_:	GPE (384)
	text:	Kolhapur
14
	label_:	ORG (383)
	text:	Maharashtra
15
	label_:	DATE (391)
	text:	2016
16
	label_:	ORG (383)
	text:	HTML
17
	label_:	DATE (391)
	text:	Less than 1 year
18
	label_:	PERSON (380)
	text:	Linux
19
	label_:	DATE (391)
	text:	Less than 1 year
20
	label_:	ORG (383)
	text:	MICROSOFT
21
	label_:	PERSON (380)
	text:	ACCESS
22
	label_:	PERSON (380)
	text:	C++
23
	label_:	PERSON (380)
	text:	Java
24
	label_:	ORG (383)
	text

##### Questions: What is your first impression of spacy's NER based on the results above? Does it seem accurate/powerfull?
##### Does it make many mistakes? Do some entity types seem more accurate than others? 
1. Good, it seems accurate.
2. Yes. For programm expertise, some are a product but also a company (not very accurate). Programming languages are not that accurately classified. Dates are classified very accurate.

Now as a comparison, we will list the entities contained in the resume's original annotated training data (remember, the existing annotations were created by a human-annotator, and not predicted by a machine like the entities predicted above) 

Complete the code below to do the following: 
* Access the 'entities' list of the example resume you chose, loop through the entities and print them out. 
* *Tip: one entity in the list is a tuple with the following structure: (12,1222,"label") where the first element is the start index of the entity in the resume text, the second element is the end index, and the third element is the label.
* Use this Tip to print out a formatted list of entities 



In [0]:
## TODO print original entities for one resume
res = resumes[0]
restext = res[0]
labeled_ents = res[1]['entities']
print(labeled_ents)
## TODO print out formatted list of entity labels and text
for ent in labeled_ents:
    print("{} {}".format(ent[2], restext[ent[0]:ent[1]]))

[[1155, 1199, 'Email Address'], [1143, 1240, 'Links'], [743, 1141, 'Skills'], [729, 733, 'Graduation Year'], [675, 703, 'College Name'], [631, 673, 'Degree'], [625, 630, 'Graduation Year'], [614, 623, 'College Name'], [606, 612, 'Degree'], [438, 454, 'Companies worked at'], [104, 148, 'Email Address'], [62, 68, 'Location'], [0, 14, 'Name']]
Email Address indeed.com/r/Afreen-Jamadar/8baf379b705e37c6
Links https://www.indeed.com/r/Afreen-Jamadar/8baf379b705e37c6?isid=rex-download&ikw=download-top&co=IN
Skills Database (Less than 1 year), HTML (Less than 1 year), Linux. (Less than 1 year), MICROSOFT
ACCESS (Less than 1 year), MICROSOFT WINDOWS (Less than 1 year)

ADDITIONAL INFORMATION

TECHNICAL SKILLS:

• Programming Languages: C, C++, Java, .net, php.
• Web Designing: HTML, XML
• Operating Systems: Windows […] Windows Server 2003, Linux.
• Database: MS Access, MS SQL Server 2008, Oracle 10g, MySql.
Graduation Year 2016
College Name Shivaji University Kolhapur 
Degree Bachelor of Engg i

As we already know, the annotated entities in the training data are different than the entities spacy can recognize with it's pretrainied models, so we need to train a custom NER model. We will get started with that now. 

##### Prepare Training Data for NER model training
We need to do some more preprocessing of our training data before we can train our model.

Remember the entity labels you chose in part 1 of the challenge? We will be training a model to predict those entities.
As a first step, we will gather all resumes that contain at least one training annotation for those entities.

Complete and execute the code below to gather your training data. 

In [0]:
##TODO Store the entity labels you want to train for as array in chosen_entity_labels
chosen_entity_labels = ['Name', 'Degree', 'Designation']

## this method gathers all resumes which have all of the chosen entites above.
def gather_candidates(dataset, entity_labels):
    candidates = list()
    for resume in dataset:
        res_ent_labels = list(zip(*resume[1]["entities"]))[2]
        if set(entity_labels).issubset(res_ent_labels):
            candidates.append(resume)
    return candidates
## TODO use the gather candidates methods and store result in training_data variable
training_data = gather_candidates(resumes, chosen_entity_labels)
print("Gathered {} training examples".format(len(training_data)))

Gathered 572 training examples


Now we have those training examples which contain the entities we are interested in. Do you have at least a few hundred examples? If not, you might need to re-think the entities you chose or try just one or two of them and re-run the notebooks. It is important that we have several hundred examples for training (e.g. more than 200. 3-500 is better). 

##### Remove other entity annotations from training data
Now that we have our training data, we want to remove all but relevant (chosen) entity annotations from this data, so that the model we train will only train for our entities. Complete and execute the code below to do this. 

In [0]:
## filter all annotation based on filter list
def filter_ents(ents, filter):
    filtered = [ent for ent in ents if ent[2] in filter]
    return filtered

## TODO use method above to remove all but relevant (chosen) entity annotations and store in X variable 
X = [[dat[0], dict(entities=filter_ents(dat[1]['entities'], chosen_entity_labels))] for dat in training_data]
#print(X)

##### Remove resumes that cause errors in spacy
Depending on what entities you chose, some of the resumes might cause errors in spacy. We don't need to get into details as to why, suffice to say it has to do with whitespace and syntax in the entity annotations. If these resumes are not removed from our training data, spacy will throw an exception during training, so we need to remove them first. 

We will use the remove_bad_data function below to do this. This function does the following:
* calls train_spacy_ner with debug=True and n_iter=1. This causes spacy to process the documents one-by-one, and gather the documents that throw an exception in a list of "bad docs" which it returns. 
* You will complete the function to remove any baddocs (returned by remove_bad_data) from your training data list. 

You may or may not have any bad documents depending on the entities you chose. In any case, there should not be more than a dozen or so bad docs.  

In [0]:
from spacy_train_resume_ner import train_spacy_ner

def remove_bad_data(training_data):
    model, baddocs = train_spacy_ner(training_data, debug=True, n_iter=1)
    ## training data is list of lists with each list containing a text and annotations
    ## baddocs is a set of strings/resume texts.
    ## TODO complete implementation to filter bad docs and store filter result (good docs) in filtered variable
    filtered = [data for data in training_data if data[0] not in baddocs]
    print("Unfiltered training data size: ",len(training_data))
    print("Filtered training data size: ", len(filtered))
    print("Bad data size: ", len(baddocs))
    return filtered

## call remove method. It may take a few minutes for the method to complete.
## you will know it is complete when the print output above. 
X = remove_bad_data(X)

Created blank 'en' model
Exception thrown when processing doc:
("Neeraj Dwivedi\nSenior Sales Executive - Kansai Nerolac Paints Ltd\n\nMumbai, Maharashtra - Email me on Indeed: indeed.com/r/Neeraj-Dwivedi/8f053ed44cdef8b2\n\nWORK EXPERIENCE\n\nSenior Sales Executive\n\nKansai Nerolac Paints Ltd -  Mumbai, Maharashtra -\n\nAugust 2017 to Present\n\nIncreased the sales from -22% to +6% through existing dealers and prospecting new dealers to\nachieve the target within 6 months of joining.\n• Maintaining excellent relations with dealers to increase revenue by 10%.\n• Management of sales team (up to 3 members)\n• Achieved monthly targets amounting to 40 - 60 lakhs monthly with timely collection.\n• Maintain accounts clarity with dealers and ensure that the credit notes reach the dealers on\ntime.\n• Business development by conducting regular meetings with influencers such as architects,\nbuilding contractors, housing societies, painters.\n• Ensuring proper visibility of the products at vari

##### Question: How many bad docs did you have? What is the size of your new (filtered) training data? 
2 bad docs. size of filtered training data: 570

##### Train/Test Split
Now before we train our model, we have to split our available training data into training and test sets. Splitting our data into train and test (or holdout) datasets is a fundamental technique in machine learning, and essential to avoid the problem of overfitting.
Before we go on, you should get a grasp of how train/test split helps us avoid overfitting. Please take the time now to do a quick web search on the topic. There are many resources available. You should search for "train test validation overfitting" or some subset of those terms.

Here are a few articles to start with:
* https://machinelearningmastery.com/a-simple-intuition-for-overfitting/
* https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation (Note: you are free to install scikit learn and use the train_test_split method documented here, but it is not necessary. It is the concept that is important)

##### Question: What is overfitting and how does doing a train/test split help us avoid overfitting when training our models? Please answer in your own words. 
Overfitting is basically the effect of an algorithm approximating too close to the given (observed) training data. As a model should not only perform well on the observed data (the set used for training) but also unobserved data, the data set is instead split to fight overfitting and so that the model can be evaluated with exclusive non-training data.

Now that we understand why we do a train/test split, we will write some code that splits our data into train and test sets. Usually we want around 70-80% of the data for train, and the rest for test. 
##### TODO: Complete the code below to create a train and test dataset

In [0]:
##TODO complete the implementation  of the train test split function below
def train_test_split(X,train_percent):
    train_size = int(len(X) * (train_percent/100))
    train = X[:train_size]
    test = X[train_size:]
    return train, test
print(f'len(X) before train_test_split(...) call: {len(X)}')
## TODO chose train size percent and call train test split, storing results in "train" and "test" variables.
train, test = train_test_split(X, 75)
## TODO use python assert to assert that the size of train and test sets add up to the size of all the data 
assert len(train) + len(test) == len(X), "len: train + test != X"

len(X) before train_test_split(...) call: 570


##### Train a spacy ner model with our training data
OK, now it is (finally) time to train our own custom NER model using spacy. Because our training data has been preprocessed to only include annotations for the entities we are interested in, the model will only be able to predict/extract those entities. 
*Depending on your computer, this step may take a while.* Training 20 epochs (iterations) using 480 training examples takes around 10 minutes on my machine (core i7 CPU). You will see output like *Losses {'ner':2342.23342342}* after each epoch/iteration. The default number of iterations is 20, so you will see this output 20 times. When this step is done, we will use the trained ner model to perform predictions on our test data in our test set.  

In [0]:
## run this code to train a ner model using spacy
custom_nlp,_= train_spacy_ner(train,n_iter=20)

Created blank 'en' model
Losses {'ner': 29299.86893881361}
Losses {'ner': 42117.45910644377}
Losses {'ner': 41704.680364185144}
Losses {'ner': 35565.431683719806}
Losses {'ner': 24458.484032785273}
Losses {'ner': 11629.156451216382}
Losses {'ner': 16113.010286056502}
Losses {'ner': 13553.540555231855}
Losses {'ner': 5804.508448688644}
Losses {'ner': 5981.521323172654}
Losses {'ner': 7400.889933400929}
Losses {'ner': 4259.968566353387}
Losses {'ner': 9207.026820985502}
Losses {'ner': 4695.298048112723}
Losses {'ner': 4398.978089769973}
Losses {'ner': 3561.5410471655273}
Losses {'ner': 4553.55782546678}
Losses {'ner': 3984.887683861325}
Losses {'ner': 3095.2879098874714}
Losses {'ner': 3008.7302642196314}


##### Inspect NER predictions on one sample resume
Now that we have a trained model, let's see how it works on one of our resumes. 

In [0]:
## TODO fetch one resume out of our test dataset and store to the "resume" variable
resume = test[3]
## TODO create a spacy doc out of the resume using our trained model and save to the "doc" variable 
doc = custom_nlp(resume[0])

Now we will output the predicted entities and the existing annotated entities in that doc

In [0]:
## TODO output label and text of predicted entities (in "ents" variable of the spacy doc created above)
print("PREDICTED:")
for ent in sorted(doc.ents, key=lambda x: str(x)):
    print("{:15}: {}".format(ent.label_, ent))
print()
## TODO output labeled entities (in "entities" dictionary of resume)
print("LABELED:")
for ent in sorted(resume[1]['entities'], key=lambda x: resume[0][x[0]:x[1]]):
     print("{:15}: {}".format(ent[2], resume[0][ent[0]:ent[1]]))

PREDICTED:
Degree         : Bachelor of Engineering in Engineering
Name           : Kavya U.
Degree         : Master of Science in Technology in Technology
Designation    : Network Ops Associate
Designation    : Network Ops Associate

LABELED:
Degree         : Bachelor of Engineering in Engineering
Name           : Kavya U.
Degree         : Master of Science in Technology in Technology

School Of Information Sciences
Designation    : Network Ops Associate



#### Evaluation Metrics for NER
Now that we can predict entities using our trained model, we can compare our predictions with the original annotations in our training data to evaluate how well our model performs for our task. The original annotations have been annotated manually by human annotators, and represent a "Gold Standard" against which we can compare our predictions. 

For most classification tasks, the most common evaluation metrics are:
* accuracy
* precision
* recall
* f1 score

In order to understand these metrics, we need to understand the following concepts:
* True positives - How many of the predicted entities are "true" according to the Gold Standard? (training annotation) 
* True negatives - How many entities did the model not predict which are actually not entities according to the Gold Standard?
* False positives - How many entities did the model predict which are NOT entities according to the Gold Standard?  
* False negatives - How many entities did the model "miss" - e.g. did not recognize as entities which are entities according to the Gold Standard? 

Before we go on, it is important that you understand true/false positives/negatives as well as the evaluation metrics above. Take some time now to research the web in order to find answers to the following questions:

##### Question: How are the evaluation metrics above defined in the context of evaluating Machine Learning models? How do they relate to True/False Positives/Negatives above? Please provide an intuitive description as well as the mathmatical formula for each metric. 
*Answers here*


##### Calculating Metrics based on token-level annotations or full entity-level. 
The concepts above are our first step toward understanding how to evaluate our model effectively. However, in NER, we need to take into account that we can calculate our metrics either based on all tokens (words) found in the document, or only on the entities found in the document.  

##### Token-Level evaluation. 
Token level evaluation evaluates how accurately did the model tag *each individual word/token* in the input. In order to understand this, we need to understand something called the "BILUO" Scheme (or BILOU or BIO). The spacy docs have a good reference. Please read and familiarize yourself with BILUO. 

https://spacy.io/api/annotation#biluo

Up to now, we have not been working with the BILUO scheme, but with "offsets" (for example: (112,150,"Email") - which says there is an "Email" entity between positions 112 and 150 in the text). We would like to be able to evaluate our models on a token-level using BILUO - so we need to convert our data to BILUO. Fortunately, Spacy provides a helper method to do this for us.

*Execute the code below to see how our "Gold Standard" and predictions for our example doc above look in BILUO scheme.* 
Note: some of the lines might be ommited for display purposes. 

In [0]:
from spacy.gold import biluo_tags_from_offsets
import pandas as pd
from IPython.display import display, HTML

## returns a pandas dataframe with tokens, prediction, and true (Gold Standard) annotations of tokens
def make_bilou_df(nlp,resume):
    """
    param nlp - a trained spacy model
    param resume - a resume from our train or test set
    """
    doc = nlp(resume[0])
    bilou_ents_predicted = biluo_tags_from_offsets(doc, [(ent.start_char,ent.end_char,ent.label_)for ent in doc.ents])
    bilou_ents_true = biluo_tags_from_offsets(doc,
                                                   [(ent[0], ent[1], ent[2]) for ent in resume[1]["entities"]])

    
    doc_tokens = [tok.text for tok in doc]
    bilou_df = pd.DataFrame()
    bilou_df["Tokens"] = doc_tokens
    bilou_df["Tokens"] = bilou_df["Tokens"].str.replace("\\s+","") 
    bilou_df["Predicted"] = bilou_ents_predicted
    bilou_df["True"] = bilou_ents_true
    return bilou_df

## TODO call method above with a resume from test set and store result in bilou_df variable.
bilou_df = make_bilou_df(custom_nlp, resume)
display(bilou_df)  

,Tokens,Predicted,True
0,Kavya,B-Name,B-Name
1,U.,L-Name,L-Name
2,,O,O
3,Network,B-Designation,O
4,Ops,I-Designation,O
5,Associate,L-Designation,O
6,-,O,O
7,Accenture,O,O
8,,O,O
9,Bengaluru,O,O


Based on this output, it should be very easy to calculate a token-level accuracy. We simply compare the "Predicted" to "True" columns and calculate what percentage are the same. 

In [0]:
## TODO bilou_df is a pandas dataframe. Use pandas dataframe api to get a subset where predicted and true are the same. 
same_df = bilou_df[bilou_df["Predicted"] == bilou_df["True"]]
## accuracy is the length of this subset divided by the length of bilou_df
accuracy = same_df.shape[0]/bilou_df.shape[0]
print("Accuracy on one resume: ", accuracy)

Accuracy on one resume:  0.96398891966759


The accuracy might seem pretty good... if it is not 100%, then let's print out those tokens where the model predicted something different than the gold standard by running the code below. 

Note - if your score on one doc is 100%, pick another document and re-run the last few cells above. 

In [0]:
## TODO find all rows in bilou_df where "Predicted" not equal to "True" column. 
diff_df = bilou_df[bilou_df["Predicted"] != bilou_df["True"]]
display(diff_df)

,Tokens,Predicted,True
3,Network,B-Designation,O
4,Ops,I-Designation,O
5,Associate,L-Designation,O
35,Network,B-Designation,-
36,Ops,I-Designation,-
37,Associate,L-Designation,-
38,,O,-
229,Technology,L-Degree,I-Degree
230,,O,I-Degree
231,School,O,I-Degree


Now let's calculate the accuracy on all our test resumes and average them for an accuracy score. 

Please complete the code below to report an accuracy score on our test resumes

In [0]:
import numpy as np
doc_accuracy = []
for res in test:
    resume_df = make_bilou_df(custom_nlp, res)
    same_df = resume_df[resume_df["Predicted"] == resume_df["True"]]
    doc_accuracy.append(same_df.shape[0]/resume_df.shape[0])

## TODO calculate mean/average of doc_accuracy (Tip: use numpy!)
total_acc = np.mean(doc_accuracy)
print("Accuracy (all resumes): ", total_acc)

Accuracy (all resumes):  0.9769256245915646


##### Question: how does the model perform on token-level accuracy? What did it miss? In those cases where the predictions didn't match the gold standard, were the predictions plausible or just "spurious" (wrong)? 
They look plausible to me. See for yourself:

In [0]:
import numpy as np
template = make_bilou_df(custom_nlp, test[0])
bad_rows = pd.DataFrame(data=None, columns=template.columns, index=None)
for res in test[:5]:
    resume_df = make_bilou_df(custom_nlp, res)
    diff_df = resume_df[resume_df["Predicted"] != resume_df["True"]]
    #display(diff_df) 
    bad_rows = bad_rows.append(diff_df, ignore_index=True)


display(bad_rows) 

,Tokens,Predicted,True
0,Chief,B-Designation,-
1,Coordinator,I-Designation,-
2,of,I-Designation,-
3,LEAR,L-Designation,-
4,,O,-
5,Chief,B-Designation,-
6,Coordinator,I-Designation,-
7,of,I-Designation,-
8,LEAR,L-Designation,-
9,,O,-


##### Question: What might the advantages and disadvantages be of calculating accuracy on token-level? Hint: think about a document with 1000 tokens where only 10 tokens are annotated as entities. What might the accuracy be on such a document?  

##### Entity-Level evaluation #####
Another method of evaluating the performance of our NER model is to calculate metrics not on token-level, but on entity level. There is a good blog article that describes this method. 

http://www.davidsbatista.net/blog/2018/05/09/Named_Entity_Evaluation/

The article goes into some detail, the most important part is the scenarios described in the section "Comparing NER system output and golden standard". 

##### Question: how do the first 3 scenarios described in the section "Comparing NER system output and golden standard" correlate to  true/false positives/negatives? 
*Answer here* 

##### Precision, Recall, F1 #####

Now we would like to calculate precision, recall, and f1 for each entity type we are interested in (our chosen entities). To do this, we need to understand the formulas for each. A good article for this is https://skymind.ai/wiki/accuracy-precision-recall-f1. 

##### Question: how can we calculate precision, recall and f1 score based on the information above? Please provide the formulas for each #####
*Answer here*

Now supply code below which calculates precision and recall and F1 on our test data for each entity type we are interested in. 



In [0]:
## TODO cycle through chosen_entity_labels and calculate metrics for each entity using test data
data = []
for label in chosen_entity_labels:
    ## variables to store results for all resumes for one entity type
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    for tres in test:
        ## use make_bilou_df on each resume in our test set, and calculate for each entity true and false positives,
        ## and false negatives. 
        tres_df = make_bilou_df(custom_nlp, tres)
        ## calculate true false positives and false negatives for each resume
        tp = tres_df[(tres_df["Predicted"].str.contains(label))
                     & (tres_df["Predicted"] == tres_df["True"])]
        fp = tres_df[(tres_df["Predicted"].str.contains(label))
                     & (tres_df["Predicted"] != tres_df["True"])]
        fn = tres_df[(tres_df["True"].str.contains(label))
                     & (tres_df["Predicted"] != tres_df["True"])]
        ## aggregate result for each resume to totals
        true_positives += tp.shape[0]
        false_positives += fp.shape[0]
        false_negatives += fn.shape[0]
    
    print("For label '{}' tp: {} fp: {} fn: {}".format(label,true_positives,false_positives,false_negatives))
    
    ## TODO Use the formulas you learned to calculate metrics and print them out
    if true_positives + false_negatives == 0:
        precision = 0.0
        recall = 0.0
    else:
        precision = true_positives / (true_positives + false_positives)
        recall = true_positives / (true_positives + false_negatives)
    if (precision + recall) == 0:
        f1 = 0.0
    else:
        f1 = 2 * (precision * recall / (precision + recall))
    #print("Precision: ",precision)
    #print("Recall: ",recall)
    #print("F1: ",f1)
    row = [precision,recall,f1]
    data.append(row)

## make pandas dataframe with metrics data. Use the chosen entity labels as an index, and the metric names as columns. 
metric_df = pd.DataFrame(data, index=chosen_entity_labels, columns=["Precision", "Recall", "F1"])
display(metric_df)

For label 'Name' tp: 290 fp: 0 fn: 6
For label 'Degree' tp: 549 fp: 368 fn: 120
For label 'Designation' tp: 762 fp: 663 fn: 434


,Precision,Recall,F1
Name,1.000000,0.979730,0.989761
Degree,0.598691,0.820628,0.692308
Designation,0.534737,0.637124,0.581457


Now we compute an average score for each metric. 

In [0]:
## TODO compute average metrics And print them out. Use pandas dataframe "mean" method to do this
print(f"Average precision:\t{metric_df['Precision'].mean()}")
print(f"Average recall:\t\t{metric_df['Recall'].mean()}")
print(f"Average f1:\t\t{metric_df['F1'].mean()}")

Average precision:	0.7111427423520634
Average recall:		0.812493759413237
Average f1:		0.7545087478143276


##### Question: how do the average metrics here (computed on entity-level) compare to the token-level accuracy score above? Which metric(s) would you prefer to use to evaluate the quality of your model? Why? 

Accuracy is rather shallow and not providing much insight if we have imbalanced classes. True positives, True negatives, False positives and False negatives give a deeper understanding for the classification situation. On the other hand, evaluating on entity-level is more meaningful to me, as we intuively want to classify entities and not tokens.

We are almost Done with part II! We just need to save our BILUO training data for reuse in Part III. 
For part III we are using flair and loading our data from a .csv file into a flair "Corpus". This is described here:

https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_6_CORPUS.md#reading-your-own-sequence-labeling-dataset

We need to create "train" and "test" .csv files using our train and test dataset which corresponds to the format described above. This format is one line containing (minimally) a text token and a NER Tag. These should be separated by whitespace. 

##### "Sentences" #####
Flair works with "Sentences" which is a list of tokens. If we simply write out our csv with one line for every token in our dataset, we will have 1 giant sentence with many thousands of words.. This is not what we want. 
We would like to partition our data so that we have a list of "Sentences" - corresponding to our intuition for a sentence - a sequence of words that belong together and is not all to long, usually separated by some punctuation. 
When we create our .csv strings/files, we need to do so so that they represent a list of sentences, each sentence consisting of a list of tokens/tags (each token/tag being one line in our csv). 

***To do this, create a blank newline after each sentence.*** 

How you split your resume data into sentences is your decision however.  

Some starter code is provided below. Complete it to create two strings which you will save as .csv. Each file will be a (long) list of token/tag lines, with sentences separated by newlines. 

In [0]:
def bilou_for_flair(nlp, train, test):
    """
    make .csv strings from train and test for use in flair
    """    
    print("Make bilou dfs... ", end='')
    ## makes a list of pandas dataframes, one for each resume. 
    training_data_as_bilou = [make_bilou_df(nlp,res) for res in train]
    test_data_as_bilou = [make_bilou_df(nlp,res) for res in test]
    print("OK")
    ## strings to return
    training_file = ""
    test_file = ""
    print("Prepare training data csv string... ", end='')
    for idx, df_ in enumerate(training_data_as_bilou):
        df = df_.copy()
        ## TODO - remove unwanted whitespace and/or newline token rows from dataframe
        df['Tokens'] = df['Tokens'].map(lambda x: x.strip())
        df = df[df['Tokens'] != '']
        # remove 'Predicted'-column, only need "a text token and a NER Tag"
        df = df.drop(['Predicted'], 1)
        ## You can use df.to_csv below to create a csv for your data
        ## as_csv = df.to_csv(None,sep=" ",encoding="utf-8",index=False,header=False,line_terminator="\n")
        as_csv = df.to_csv(None,sep=" ",encoding="utf-8",index=False,header=False,line_terminator="\n")
        ## TODO - insert newlines after each "sentence"
        as_csv2 = ''
        for line in as_csv.splitlines():
            if line[0] in ['.', '!', '?']:
                line += '\n'
            as_csv2 += line + '\n'
        ## training_file = TODO append csv string for each resume to big training file (dont forget newline after each)
        training_file += as_csv2 + '\n'
    print("OK")
    print("Prepare test data csv string... ", end='')
    for idx, df_ in enumerate(test_data_as_bilou):
        df = df_.copy()
        ## TODO - remove unwanted whitespace and/or newline token rows from dataframe
        df['Tokens'] = df['Tokens'].map(lambda x: x.strip())
        df = df[df['Tokens'] != '']
        # remove 'Predicted'-column, only need "a text token and a NER Tag"
        df = df.drop(['Predicted'], 1)
        ## You can use df.to_csv below to create a csv for your data
        ## as_csv = df.to_csv(None,sep=" ",encoding="utf-8",index=False,header=False,line_terminator="\n")
        as_csv = df.to_csv(None,sep=" ",encoding="utf-8",index=False,header=False,line_terminator="\n")
        ## TODO - insert newlines after each "sentence"
        as_csv2 = ''
        for line in as_csv.splitlines():
            if line[0] in ['.', '!', '?']:
                line += '\n'
            as_csv2 += line + '\n'
        ## test_file = TODO append csv string for each resume to big training file (dont forget newline after each)
        test_file += as_csv2 + '\n'
    print("OK")
    return training_file, test_file

training,test = bilou_for_flair(custom_nlp,train,test)
## TODO save training and test as .csv (wherever you want)
print('Write to file... ', end='')
with open("data/training_bilou.txt", 'w', encoding="utf-8") as f:
    f.write(training)
with open("data/test_bilou.txt", 'w', encoding="utf-8") as f:
    f.write(test)
print("OK")

Make bilou dfs
Done!
Write to file...


Now let's load the data we persisted with flair before we go on

In [12]:
# imports 
from flair.datasets import Corpus
from flair.data_fetcher import NLPTaskDataFetcher

# folder where training and test data are
data_folder = 'data'
# your training file name
train_file = 'training_bilou.txt'

# your training file name
test_file = 'test_bilou.txt'

columns = {
            0: "Tokens",
            1: "Predicted",
            2: "True"
          }## TODO how is your file structured? {}

## Now load our csv into flair corpus
corpus = NLPTaskDataFetcher.load_column_corpus(data_folder, column_format=columns,
                                               train_file=train_file,
                                               test_file=test_file)
print(corpus)

2019-06-19 01:26:27,121 Reading data from data
2019-06-19 01:26:27,126 Train: data/training_bilou.txt
2019-06-19 01:26:27,130 Dev: None
2019-06-19 01:26:27,134 Test: data/test_bilou.txt


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated function (or staticmethod) load_column_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:312: DeprecationWarning: Call to deprecated function (or staticmethod) read_column_data. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  train_file, column_format
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:318: DeprecationWarning: Call to deprecated function (or staticmethod) read_column_data. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  test_file, column_format


Corpus: 9401 train + 1044 dev + 3100 test sentences


If you could load the corpus without error, you are ready to go on to part 3, where we will work with flair nlp!

If you have something like between 5 and 10k training sentences, and roughly a third of that as